In [10]:
import os 
import re
import glob
import pandas as pd
from tqdm import tqdm
from datafc.trans.model import TransformationModel

udata = TransformationModel()
dataPath = '/beegfs/ws/0/s9908945-format_transformation/s9908945-paper/'
fuzzyGenerators = glob.glob(f'{dataPath}configuration/fuzzyGenerators/*.py')
types = [os.path.splitext(os.path.basename(fuzzyGenerator))[0] for fuzzyGenerator in fuzzyGenerators]

dfs = {}
for type in types:
    df = pd.read_parquet(f'{dataPath}data/Correction/test/{type}.parquet')
    dfs[type] = df

In [14]:
examples = {
    'Address': {
        "number street, city, state postalCode, country": ['416 South University Boulevard, Norman, Madhya Pradesh 73069, India', '8 Elmwood Drive, Rutland, Ribnica na Pohorju 05701, Slovenia', '118 Pearl Street, Manchester, Natore 06040, Bangladesh'],
        "number street, city, postalCode, state, country": ['396 Hartford Road, Manchester, 06040, Punjab, Pakistan', '130 West Brown Road, Mesa, 85201, Guam, United States', '1718 Underwood Road, Gambrills, 21054, Kalangala, Uganda'],
        "number street, city, postalCode, state": ['6460 Lacrosse Lane, Glen Burnie, 21061, Telšių apskritis', '7400 West Arrowhead Clubhouse Drive, Glendale, 85308, Chichaoua', '8757 Lamar Circle, Arvada, 80003, Ulsan-gwangyeoksi'],
        "number street, city, state postalCode": ['37675 Fremont Boulevard, Fremont, Al Janūb 94536', '1917 Whitaker Street, Savannah, Ad Dawḩah 31401', '408 Pine Street, Bloomingdale, Saint George 31302'],
        "number street, city, country": ['8666 Iris Street, Arvada, Morocco', '73 Twin Hills Drive, Longmeadow, Tanzania, United Republic of', '12 Fletcher Lane, Shelburne, Azerbaijan'],
        "number street, city, state": ['36043 Soapberry Commons, Fremont, Tolima', '25 Edwards Street, Manchester, Aḑ Ḑāli‘', '13415 Panama City Beach Parkway, Panama City Beach, Savannakhét'],
        "number street, city": ['8800 Northwest 92nd Street, Oklahoma City', '3505 Albee Drive, Nashville', '903 East 20th Avenue, Anchorage'],
        "number street": ['4714 Narrow Lane Road', '549 Elysian Fields Road', '5836 Red Barn Road'],
        "street number, city": ['Spy Glass Drive 1102, Arnold', 'Amity Lane 741, Montgomery', 'North 56th Avenue 13066, Glendale'],
        "street number, city, postalCode": ['West Muhammad Ali Boulevard 3825, Louisville, 40212', 'Dutchmans Court 718, Nashville, 37076', 'Sandra Grace Road 9222, Southport, 32409'],
        "street number, city, state, postalCode, country": ['Paseo Place 22825, Hayward, Est, 94541, Burkina Faso', 'North 54th Avenue 5065, Glendale, Tartumaa, 85301, Estonia', 'Connecticut Avenue Northwest 1627, Washington, Kitgum, 20009, Uganda'],
        "street number, city, state, country": ['Pennington PIace 9030, Montgomery, Western, Papua New Guinea', 'Falcon Nest Court 1489, Arnold, Lombardia, Italy', 'Thistle Down Lane 3109, Nashville, Vóreio Aigaío, Greece']
    },
    'Continent': {
        "name" : ['Africa', 'Asia', 'Europe'],
        "code" : ['AF', 'AS', 'EU']
    }, 
    'Country': {
        "alpha_2": ['NR', 'GB', 'UG'],
        "alpha_3": ['UKR', 'GMB', 'COD'],
        "name": ['Aruba', 'Uzbekistan', 'Montserrat'],
        "official_name": ['French Republic', 'Republic of Burundi', "Republic of Côte d'Ivoire"]
    },
    'Date': {
        'DD.MM.YYYY': ['11.05.1982', '13.12.1969', '01.01.1915'],
        'DD.MM.YY': ['07.02.58', '20.01.39', '07.02.61'],
        'DD.MN.YYYY': ['23.December.1955', '09.October.1947', '15.May.1991'],
        'DD.MN.YY': ['21.September.37', '31.May.69', '31.December.86'],
        'DD.MA.YYYY': ['19.Dec.1858', '29.Jun.1961', '13.Jan.1876'],
        'DD.MA.YY': ['20.May.06', '02.Mar.46', '18.Sep.21'],
        'DD.MM.YYYY HH:MIN:SS': ['04.01.1989 10:17:31', '12.07.1930 10:17:02', '05.08.1973 10:17:58'],
        'DD.MM.YY HH:MIN:SS': ['08.03.11 10:17:35', '05.06.33 10:17:40', '21.09.09 10:17:18'],
        'DD.MN.YYYY HH:MIN:SS': ['18.August.1952 10:17:04', '13.May.1883 10:17:28', '22.July.1932 10:17:17'],
        'DD.MN.YY HH:MIN:SS': ['18.March.52 10:17:14', '22.January.57 10:17:25', '12.August.94 10:17:56'],
        'DD.MA.YYYY HH:MIN:SS': ['15.Jan.1869 10:17:35', '06.Feb.1955 10:17:07', '05.Feb.1870 10:17:52'],
        'DD.MA.YY HH:MIN:SS': ['18.Feb.93 10:17:22', '27.Apr.87 10:17:30', '01.Jan.81 10:17:26'],
        'WD, DD.MM.YYYY': ['Wednesday, 22.03.2006', 'Wednesday, 24.08.1887', 'Monday, 19.10.1981'],
        'WD, DD.MM.YY': ['Monday, 16.09.40', 'Wednesday, 05.07.06', 'Saturday, 28.08.76'],
        'WD, DD.MN.YYYY': ['Tuesday, 16.March.1993', 'Sunday, 27.November.1938', 'Tuesday, 07.December.1858'],
        'WD, DD.MN.YY': ['Monday, 08.February.71', 'Saturday, 05.June.15', 'Monday, 14.May.23'],
        'WD, DD.MA.YYYY': ['Monday, 22.Dec.1902', 'Saturday, 21.Feb.2009', 'Wednesday, 26.Jun.1991'],
        'WD, DD.MA.YY': ['Sunday, 16.May.47', 'Saturday, 19.Aug.43', 'Thursday, 14.Apr.94'],
        'WD, DD.MM.YYYY HH:MIN:SS': ['Tuesday, 31.01.1832 10:17:50', 'Wednesday, 09.07.1856 10:17:57', 'Wednesday, 27.06.1860 10:17:30'],
        'WD, DD.MM.YY HH:MIN:SS': ['Wednesday, 16.11.87 10:17:54', 'Sunday, 01.08.10 10:17:43', 'Sunday, 21.07.95 10:17:04'],
        'WD, DD.MN.YYYY HH:MIN:SS': ['Friday, 23.April.1897 10:17:01', 'Tuesday, 17.January.1893 10:17:55', 'Monday, 27.April.2015 10:17:20'],
        'WD, DD.MN.YY HH:MIN:SS': ['Thursday, 02.May.40 10:17:44', 'Sunday, 02.May.71 10:17:53', 'Thursday, 23.July.70 10:17:17'],
        'WD, DD.MA.YYYY HH:MIN:SS': ['Sunday, 10.Nov.1844 10:17:43', 'Wednesday, 26.Sep.2001 10:17:25', 'Saturday, 14.Jan.1865 10:17:48'],
        'WD, DD.MA.YY HH:MIN:SS': ['Monday, 02.May.81 10:17:24', 'Saturday, 07.Dec.13 10:17:47', 'Saturday, 22.Dec.79 10:17:29'],
        'DD/MM/YYYY': ['03/08/1830', '17/03/1936', '09/09/1844'],
        'DD/MM/YY': ['09/08/93', '16/02/86', '13/10/25'],
        'DD/MN/YYYY': ['23/January/1997', '30/June/1986', '20/October/1854'],
        'DD/MN/YY': ['05/April/28', '03/December/91', '15/June/09'],
        'DD/MA/YYYY': ['05/Jun/1988', '20/Jan/1997', '11/Aug/1999'],
        'DD/MA/YY': ['02/Aug/91', '21/Aug/58', '01/Oct/41'],
        'DD/MM/YYYY HH:MIN:SS': ['27/05/1951 10:17:30', '23/05/1881 10:17:10', '05/08/1957 10:17:21'],
        'DD/MM/YY HH:MIN:SS': ['12/11/81 10:17:54', '29/01/58 10:17:17', '25/02/52 10:17:27'],
        'DD/MN/YYYY HH:MIN:SS': ['13/June/1903 10:17:40', '24/June/2013 10:17:48', '12/March/1880 10:17:46'],
        'DD/MN/YY HH:MIN:SS': ['03/April/62 10:17:32', '30/January/90 10:17:04', '27/April/53 10:17:32'],
        'DD/MA/YYYY HH:MIN:SS': ['22/Jan/1975 10:17:56', '21/Feb/1994 10:17:24', '12/Sep/1900 10:17:39'],
        'DD/MA/YY HH:MIN:SS': ['07/Dec/17 10:17:25', '27/Jun/06 10:17:09', '02/Nov/99 10:17:46'],
        'WD, DD/MM/YYYY': ['Sunday, 20/12/1959', 'Sunday, 22/04/1956', 'Tuesday, 13/01/1942'],
        'WD, DD/MM/YY': ['Wednesday, 10/11/10', 'Thursday, 11/05/23', 'Thursday, 11/06/59'],
        'WD, DD/MN/YYYY': ['Sunday, 30/August/1998', 'Friday, 13/December/1907', 'Saturday, 26/August/1882'],
        'WD, DD/MN/YY': ['Thursday, 19/September/29', 'Tuesday, 08/May/27', 'Thursday, 01/July/97'],
        'WD, DD/MA/YYYY': ['Sunday, 04/Aug/1861', 'Wednesday, 21/Sep/1955', 'Monday, 04/Mar/2019'],
        'WD, DD/MA/YY': ['Friday, 23/Feb/45', 'Sunday, 21/Sep/30', 'Thursday, 17/Aug/23'],
        'WD, DD/MM/YYYY HH:MIN:SS': ['Saturday, 31/03/1900 10:17:29', 'Tuesday, 19/09/1967 10:17:37', 'Sunday, 19/06/1921 10:17:50'],
        'WD, DD/MM/YY HH:MIN:SS': ['Wednesday, 08/09/21 10:17:33', 'Tuesday, 17/01/54 10:17:24', 'Monday, 06/03/71 10:17:42'],
        'WD, DD/MN/YYYY HH:MIN:SS': ['Saturday, 15/December/1866 10:17:35', 'Thursday, 27/January/1853 10:17:58', 'Friday, 09/October/1891 10:17:42'],
        'WD, DD/MN/YY HH:MIN:SS': ['Wednesday, 04/March/68 10:17:30', 'Thursday, 02/April/03 10:17:01', 'Friday, 28/June/91 10:17:36'],
        'WD, DD/MA/YYYY HH:MIN:SS': ['Monday, 23/Nov/1959 10:17:56', 'Wednesday, 18/Aug/1880 10:17:06', 'Wednesday, 28/Apr/1880 10:17:39'],
        'WD, DD/MA/YY HH:MIN:SS': ['Monday, 05/Sep/27 10:17:01', 'Tuesday, 17/Apr/17 10:17:56', 'Monday, 26/Jul/37 10:17:44'],
        'DD-MM-YYYY': ['25-04-1891', '15-09-2023', '18-02-1886'],
        'DD-MM-YY': ['13-08-45', '03-10-51', '24-04-49'],
        'DD-MN-YYYY': ['03-March-1934', '06-June-1887', '25-July-2013'],
        'DD-MN-YY': ['02-August-52', '06-July-20', '13-July-32'],
        'DD-MA-YYYY': ['27-Oct-1840', '08-Feb-1986', '24-Oct-1908'],
        'DD-MA-YY': ['07-Jun-13', '16-Jun-79', '17-Nov-30'],
        'DD-MM-YYYY HH:MIN:SS': ['01-02-1900 10:17:13', '12-02-1958 10:17:29', '07-01-1829 10:17:22'],
        'DD-MM-YY HH:MIN:SS': ['05-05-21 10:17:47', '30-04-56 10:17:57', '21-12-90 10:17:02'],
        'DD-MN-YYYY HH:MIN:SS': ['28-January-1984 10:17:33', '04-June-1979 10:17:55', '10-June-1841 10:17:09'],
        'DD-MN-YY HH:MIN:SS': ['03-October-36 10:17:25', '01-August-88 10:17:57', '17-February-82 10:17:52'],
        'DD-MA-YYYY HH:MIN:SS': ['29-Apr-2005 10:17:11', '09-Jul-1959 10:17:31', '04-May-1976 10:17:22'],
        'DD-MA-YY HH:MIN:SS': ['12-Mar-39 10:17:58', '12-Jul-36 10:17:08', '09-May-75 10:17:21'],
        'WD, DD-MM-YYYY': ['Sunday, 29-05-1988', 'Friday, 20-10-1837', 'Monday, 23-03-1908'],
        'WD, DD-MM-YY': ['Wednesday, 19-02-30', 'Wednesday, 29-03-16', 'Wednesday, 09-10-50'],
        'WD, DD-MN-YYYY': ['Tuesday, 24-December-1912', 'Sunday, 09-September-1973', 'Friday, 13-December-1907'],
        'WD, DD-MN-YY': ['Friday, 18-March-92', 'Wednesday, 07-July-47', 'Friday, 13-November-92'],
        'WD, DD-MA-YYYY': ['Tuesday, 25-Oct-1898', 'Tuesday, 05-Sep-1967', 'Monday, 05-Mar-1934'],
        'WD, DD-MA-YY': ['Sunday, 16-May-26', 'Thursday, 26-Apr-66', 'Monday, 09-Oct-17'],
        'WD, DD-MM-YYYY HH:MIN:SS': ['Saturday, 10-01-1948 10:17:43', 'Monday, 09-08-1909 10:17:42', 'Saturday, 09-08-1856 10:17:20'],
        'WD, DD-MM-YY HH:MIN:SS': ['Sunday, 17-05-31 10:17:03', 'Sunday, 24-07-38 10:17:24', 'Tuesday, 04-06-67 10:17:15'],
        'WD, DD-MN-YYYY HH:MIN:SS': ['Tuesday, 11-June-1867 10:17:14', 'Sunday, 07-March-2010 10:17:22', 'Monday, 08-July-1940 10:17:16'],
        'WD, DD-MN-YY HH:MIN:SS': ['Thursday, 29-May-58 10:17:08', 'Tuesday, 09-September-52 10:17:23', 'Friday, 11-January-57 10:17:04'],
        'WD, DD-MA-YYYY HH:MIN:SS': ['Sunday, 18-Oct-1981 10:17:35', 'Sunday, 29-Aug-1886 10:17:47', 'Tuesday, 03-Jan-1984 10:17:26'],
        'WD, DD-MA-YY HH:MIN:SS': ['Sunday, 07-Aug-88 10:17:39', 'Wednesday, 07-Jul-48 10:17:02', 'Monday, 22-Aug-27 10:17:13'],
        'MN DD, YYYY': ['June 10, 1855', 'November 24, 1955', 'March 21, 1903'],
        'MA DD, YYYY': ['Sep 30, 1864', 'Mar 25, 1989', 'Jul 27, 1825'],
        'MA. DD, YYYY': ['May. 30, 1875', 'Nov. 30, 1847', 'Apr. 05, 1885'],
        'YYYY-MM-DD': ['2015-12-23', '1975-05-27', '1998-07-14'],
        'YYYY-MN-DD': ['1846-July-23', '1980-June-05', '1957-September-12'],
        'YYYY-MA-DDMM/DD/YY': ['1954-Jul-1307/13/54', '1980-May-1605/16/80', '1951-Nov-2611/26/51'],
        'MM/DD/YYYY': ['09/01/1922', '10/02/1903', '07/24/1945'],
        'MM/DD/YY': ['09/01/22', '10/02/03', '07/24/45'],
        'WD DD, MN YYYY': ['Friday 21, September 1900', 'Monday 09, November 1857', 'Friday 04, November 1853'],
        'MM-DD-YY': ['01-25-70', '08-15-41', '02-17-75'],
        'WD, MN DD, YYYY': ['Saturday, August 17, 1895', 'Wednesday, March 19, 1947', 'Saturday, August 12, 2023'],
        'WD, DD MN YYYY': ['Wednesday, 17 January 1849', 'Friday, 17 March 1871', 'Saturday, 24 July 1926'],
        'YYYY-MM-DD HH:MIN:SS': ['2006-11-24 10:17:50', '1982-08-20 10:17:26', '1967-05-14 10:17:51'],
        'WD DD MN, YYYY': ['Thursday 30 March, 1978', 'Friday 29 December, 1961', 'Tuesday 21 June, 2022'],
        'MM/DD/YY HH:MIN:SS': ['07/03/33 10:17:24', '11/28/50 10:17:02', '02/04/39 10:17:13'],
        'WD DD MN YYYY': ['Saturday 30 September 1972', 'Saturday 01 November 1862', 'Tuesday 29 January 1889'],
        'MN DD, YYYY': ['March 21, 1948', 'March 18, 1994', 'October 28, 1964'],
        'WD DD MN YY': ['Tuesday 09 April 85', 'Friday 01 February 50', 'Friday 21 March 02'],
        'YYYY-MM-DDTHH:MIN:SS': ['1886-04-22T10:17:55', '1902-07-27T10:17:56', '1977-07-07T10:17:01'],
        'WD, DD/MN/YYYY': ['Tuesday, 08/July/1884', 'Monday, 30/July/1973', 'Sunday, 05/April/1846'],
        'MN DD YYYY': ['November 01 1991', 'January 31 1882', 'March 13 1869'],
        'WD DD-MN-YY': ['Tuesday 04-May-48', 'Wednesday 09-July-52', 'Saturday 12-April-80'],
        'MN DD, YY': ['September 30, 32', 'November 06, 17', 'April 02, 11'],
        'WD, DD-MN-YYYY': ['Friday, 08-January-2021', 'Thursday, 09-October-1930', 'Friday, 05-August-2011'],
        'MN DD, YYYY HH:MIN:SS': ['August 17, 1919 10:17:18', 'July 09, 1839 10:17:59', 'December 28, 1917 10:17:24'],
        'WD, DD MN YY': ['Tuesday, 03 April 88', 'Friday, 29 December 06', 'Monday, 08 May 99'],
        'MN DD, YYYY HH:MIN': ['November 12, 1831 10:17', 'August 02, 1937 10:17', 'May 28, 1835 10:17'],
        'WD DD-MN-YYYY HH:MIN:SS': ['Monday 07-November-2016 10:17:16', 'Saturday 26-May-1838 10:17:27', 'Tuesday 18-November-1997 10:17:18'],
        'MN/DD/YYYY': ['April/28/1989', 'August/09/1983', 'June/09/1990'],
        'WD DD MN, YYYY HH:MIN:SS': ['Saturday 07 December, 2019 10:17:47', 'Wednesday 11 March, 1891 10:17:48', 'Tuesday 07 July, 1970 10:17:08']
    },
    'Name': {
        "{forename} {surname}": ['Lisa Styles', 'Joseph Hart', 'Andrew Lamere'],
        "{surname}, {forename}": ['Taylor, Brian', 'Diaz, Sharon', 'Gordillo, Denita'],
        "{surname} {forename}": ['Berg Elsa', 'Wynn Sylvia', 'Reed Jeffrey'],
        "{surname}, {middleName}, {forename}": ['Stone, Nelson, Cleveland', 'Pruitt, Cliff, Henry', 'Anderson, Shannon, Regina'],
        "{forename} {middleName} {surname}": ['James Sara Wallace', 'Christy Jon Butler', 'Pamela Donna Noyes'],
        "{surname}, {forename} {middleName}": ['Gray, Alison Christine', 'Pepin, Esther Brenda', 'Davin, Clinton Rodney'],
        "{surname} {forename} {middleName}": ['Dalbec Charlene Amanda', 'Carrell Rafael Cynthia', 'Daniel Verna Ned'],
        "{forename} {middleInitial}. {surname}": ['Ramon C. Greathouse', 'Amanda D. Smith', 'Christopher M. Beasley'],
        "{surname}, {initial}": ['Courtney, W', 'Perdue, J', 'Hardiman, C'],
        "{surname} {middleInitial}. {forename}": ['Cain A. Ana', 'Nunez B. Marvin', 'Davis E. Rachel'],
        "{forename} {middleInitial}. {surname}": ['Virgil V. Newman', 'Milton B. Londo', 'Julian S. Brede'],
        "{surname}, {forename} {middleInitial}.": ['Sepulveda, Arnold W.', 'Carrasco, Colleen J.', 'Hussey, Michael I.'],
        "{surname} {forename} {middleInitial}": ['Stallard Jimmy K', 'Rambo Melvin D', 'Guerrero Bryon O'],
        "{forename} ({surname})": ['Joyce (Gabaldon)', 'Juan (Johnston)', 'Dave (Keating)'],
        "{forename} ({surname}) {middleName}": ['Donna (Williams) David', 'Robby (Chatfield) Irene', 'Sophie (Mitchell) Chester'],
        "{surname}, {forename} ({middleName})": ['Cozzolino, Jose (Cedric)', 'Sickles, Betty (Nicholas)', 'Etheridge, Mildred (Mary)'],
        "{surname} ({forename})": ['Silva (Charlie)', 'Meadows (Matthew)', 'Ford (Patrick)'],
        "{surname} ({forename}) {middleName}": ['Arana (Ashley) Richard', 'Blackmer (Sue) Edward', 'Wilson (Lawrence) Lloyd'],
        "{forename} - {surname}": ['Alma - Smith', 'William - Sandlin', 'Edgar - Doi'],
        "{surname} - {forename}": ['Stringer - Toby', 'Jackson - Rolf', 'Johnson - Loretta'],
        "{initial}. {surname}": ['D. Chene', 'B. Johnson', 'D. Lewis'],
        "{initial}. {surnameInitial}.": ['B. C.', 'M. G.', 'K. G.'],
        "{forename} {surnameInitial}.": ['Matthew R.', 'Ann C.', 'Paul H.']
    },
    'Phone': {
        "({areaCode}) {exchangeCode}-{subscriberNumber}": ['(380) 98649-3392', '(225) 98741-739417', '(4002) 9073-762232'],
        "+{countryCode} ({areaCode}) {exchangeCode}-{subscriberNumber}": ['+62 (23) 95-449352', '+929 (0694) 33-162602', '+32 (3) 77-59544'],
        "+{countryCode}{areaCode}{exchangeCode}{subscriberNumber}": ['+912556664147627', '+7275125555', '+4641084020946'],
        "0{areaCode}{exchangeCode}{subscriberNumber}": ['000027515126', '057579289105', '0150254479'],
        "{areaCode}-{exchangeCode}-{subscriberNumber}": ['65-955-719120', '4248-47615-67303', '29-482-235911'],
        "{areaCode}.{exchangeCode}.{subscriberNumber}": ['5095.8844.10707', '0119.202.92349', '1199.14.890289'],
        "{areaCode} {exchangeCode} {subscriberNumber}": ['14 83029 33970', '80 32351 700628', '49 7118 698067'],
        "({areaCode}) {exchangeCode}{subscriberNumber}": ['(0043) 54975164', '(073) 5782542', '(1410) 06484687'],
        "{countryCode} {areaCode} {exchangeCode} {subscriberNumber}": ['789 343 6964 4306', '69 497 55277 93476', '012 5 67 9183'],
        "+{countryCode} ({areaCode}) {exchangeCode}-{subscriberNumber}": ['+667 (7) 007-534992', '+79 (5871) 38559-51380', '+30 (2) 878-24941'],
        "{areaCode}-{exchangeCode}-{subscriberNumber} ext. {extension}": ['895-379-76351 ext. 6', '2-867-8138 ext. 397', '898-5653-5246 ext. 2'],
        "+{countryCode}.{areaCode}.{exchangeCode}.{subscriberNumber}": ['+722.5.98156.745285', '+20.2.52982.44451', '+45.30.5325.40171'],
        "{areaCode} - {exchangeCode} - {subscriberNumber}": ['2 - 97650 - 404776', '059 - 57426 - 70441', '7614 - 9732 - 84463'],
        "({areaCode}).{exchangeCode}.{subscriberNumber}": ['(9).6025.0255', '(866).59.6289', '(42).29.50791'],
        "{areaCode} ({exchangeCode}) {subscriberNumber}": ['2986 (9229) 1421', '1 (3740) 602203', '398 (2739) 976470'],
        "{areaCode}{exchangeCode}{subscriberNumber}": ['937239360970', '06545775371572', '29347849715'],
        "{areaCode}-{exchangeCode}-{subscriberNumber} x {extension}": ['0-45544-35756 x 60', '750-4485-793200 x 03', '2649-86-41321 x 28'],
        "{areaCode} {exchangeCode} {subscriberNumber} ext {extension}": ['0627 32 991380 ext 1699', '1744 0132 1733 ext 4502', '9 86659 616144 ext 13'],
        "({countryCode}) {areaCode} {exchangeCode} {subscriberNumber}": ['(81) 688 14 806409', '(53) 4769 31 7187', '(690) 5209 055 1383'],
        "+{countryCode} {areaCode} {exchangeCode} {subscriberNumber}": ['+09 1773 007 90100', '+356 412 623 73144', '+335 30 199 7644'],
        "+00{countryCode} {areaCode} {exchangeCode} {subscriberNumber}": ['+00305 87 21731 3366', '+0039 50 66182 95842', '+0066 910 51 54003'],
        "[{areaCode}].{exchangeCode}.{subscriberNumber}": ['[4532].850.78584', '[48].6384.9380', '[012].947.478847'],
        "({areaCode}) {exchangeCode} {subscriberNumber}": ['(6) 22251 3844', '(007) 57 54289', '(2509) 56636 876142']
    },
    'Sex': {
        "code" : ['F', 'M', 'D'],
        "version_1" : ['Female', 'Male', 'Diverse'],
        "version_2" : ['Woman', 'Man', 'Diverse']
    },
    'Unit': {
        "UWV": ['kJ/mol -9742', 'c 1570', 'psi 1656.0'],
        "VWU": ['1097 eur', '1251.838 pptv', '-3044 m/s']
    }
}

In [12]:
pattern = r'[^\w\s\[\]{}()\-\+:.,\/]' #UDATA seems to not working properly with special chars

def getResults(type):
    df = dfs[type]
    correct = 0
    incorrect = 0
    for idx, _ in tqdm(df.iterrows()):
        input = df['input'][idx]
        format, inputValues = input.split(' reshape: ')
        inputValues = inputValues.split(' [ROW] ')
        output = df['output'][idx]
        target = output.split(' [ROW] ')
        outputExamples = examples[type][format]
        inputValues = [re.sub(pattern, 'p', element) for element in inputValues]
        inputValues = [element.replace('ʻ', '') for element in inputValues]
        outputExamples = [re.sub(pattern, 'p', element) for element in outputExamples]
        result = udata.learn(inputValues, outputExamples)
        results = [tup[1] for tup in result[0]]
        if target == results:
            correct += 1
        else:
            incorrect += 1
    return correct, incorrect

In [13]:
for type in types:
    print(type)
    print(getResults(type))

Name


1000it [01:38, 10.14it/s]


(503, 497)
Phone


1000it [01:38, 10.17it/s]


(219, 781)
Unit


...

(0, 1000)
Sex


1000it [00:22, 43.50it/s]


(455, 545)
Date


1it [00:00,  8.23it/s]


KeyError: 'MM/DD/YY'